In [1]:
import os
import nibabel as nib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
import random

from scipy import ndimage

from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage import transform
from skimage.transform import resize
import skimage.exposure as skie

import ot

import torch
from torch import manual_seed
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torchsummary import summary
import torchvision
from torchvision import datasets, models, transforms

from torchmetrics import Accuracy

import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy


from captum.attr import IntegratedGradients, Saliency, DeepLift, DeepLiftShap, GradientShap, GuidedBackprop,  GuidedGradCam 
from captum.attr import Deconvolution, ShapleyValueSampling, Lime, KernelShap, LRP, InputXGradient, FeatureAblation
from scipy.ndimage import sobel, laplace

torch.cuda.is_available()

torch.set_num_threads(3)

def compactness(blob_labels):
    import math
    compactness=[]
    region=regionprops(blob_labels)
    for rp in region:
        area=rp.area
        #perimeter=rp.perimeter
        perimeter=rp.perimeter_crofton
        c=(4*math.pi*area)/(perimeter**2)
        compactness.append(c)
    return compactness

def get_blobs(image,lesion_size):
    """
    gets the noise image after pre processing and returns blobs with the size equal to lesion size
    image is the noise image 
    lesion_size can be an int
    """
    
    labeled, nr_objects = ndimage.label(image)
    sizes = ndimage.sum_labels(image,labeled,range(nr_objects+1))    
    mask_size=sizes!=lesion_size
    small_blobs=labeled.copy()
    remove_pixel = mask_size[small_blobs]
    small_blobs[remove_pixel] = 0 
    
    return small_blobs


def list_lesions(image,lesion_size):
    round_lesion_c=0.8 #lesions with compactness above this value are considered round
    not_round_lesion_c=0.4 #lesions with compactness bellow this value are considered not round
    
    border=2
    blur=0.75
    
    less_size=lesion_size*0.05*blur #the lesion is smaller after the blur
    
    small_blobs=get_blobs(image,lesion_size)
    rb=regionprops(small_blobs)
    round_lesions=[]
    not_round_lesions=[]
    c_list=compactness(small_blobs)
    
    for blob in range(len(c_list)):
        
        if c_list[blob]>round_lesion_c:#round lesions

            blob_img=rb[blob].image.astype(float)
            #the image is padded because when it is smoothed it increases a bit
            pad_img=np.pad(array=blob_img, pad_width=border, mode='constant', constant_values=0)
            blur_image=ndimage.gaussian_filter(pad_img, blur)
            blur_image[blur_image<0.2]=0
            energy=round(np.sum(blur_image.astype(float)),2)
            if energy<=lesion_size-less_size+1 and energy>=lesion_size-less_size-1:
                round_lesions.append([blur_image,round(c_list[blob],3)])
        
        
        elif c_list[blob]<not_round_lesion_c:# not round lesions
            blob_img=rb[blob].image.astype(float)
            #the image is padded because when it is smoothed it increases a bit
            pad_img=np.pad(array=blob_img, pad_width=border, mode='constant', constant_values=0)
            blur_image=ndimage.gaussian_filter(pad_img, blur)
            blur_image[blur_image<0.15]=0
            energy=round(np.sum(blur_image.astype(float)),2)
            if energy<=lesion_size-less_size+1 and energy>=lesion_size-less_size-1:
                not_round_lesions.append([blur_image,round(c_list[blob],3)])
            
    return round_lesions, not_round_lesions

def create_lesions(lesion_number,lesion_size,factor=1):
    #factor is the value for which we multiply the sides of the lesion
    round_lesions=[]
    not_round_lesions=[]
    s=0
    size_noise=256
    blur_radius=2

    while len(not_round_lesions)<=lesion_number or len(round_lesions)<=lesion_number:
        #create noise
        np.random.seed(s)
        noise_img=np.random.rand(size_noise,size_noise)

        #smooth noise
        imgf=ndimage.gaussian_filter(noise_img, blur_radius)

        #create binary image
        thr=threshold_otsu(imgf)
        imgf_bin=imgf>thr
        
        #morphologic changes
        erosion_image=ndimage.binary_erosion(imgf_bin)
        open_er_img=ndimage.binary_opening(erosion_image)
        erosion_image2=ndimage.binary_erosion(open_er_img)

        #images
        

        #as a result from one noise image we create several images that can be used to create the lesions
        round_lesions_open, not_round_lesions_open=list_lesions(open_er_img,lesion_size)
        round_lesions_er, not_round_lesions_er=list_lesions(erosion_image2,lesion_size)
        
        round_lesions=round_lesions+round_lesions_open+round_lesions_er
        not_round_lesions=not_round_lesions+not_round_lesions_open+not_round_lesions_er
        
        
        #print('len lists:',len(round_lesions),len(not_round_lesions))
        '''        
        plt.figure(figsize=(15,7))
        plt.subplot(2,4,1)
        plt.imshow(noise_img)
        plt.title('noise')
        plt.subplot(2,4,2)
        plt.imshow(imgf)
        plt.title('blur')
        plt.subplot(2,4,3)
        plt.imshow(imgf_bin)
        plt.title('binary img')
        plt.subplot(2,4,4)
        plt.imshow(erosion_image)
        plt.title('erosion')
        plt.subplot(2,4,5)
        plt.imshow(open_er_img)
        plt.title('open')
        plt.subplot(2,4,6)
        plt.imshow(erosion_image2)
        plt.title('erosion2')
        plt.subplot(2,4,7)
        plt.imshow(dilated_image)
        plt.title('dilation')
        break
        '''
        
        s+=1
    print(f'round lesions: {len(round_lesions)}/{lesion_number} === not round lesions {len(not_round_lesions)}/{lesion_number}')
    print(f'number of seeds used: {s-1}')
    print(f'the lesions have size between {lesion_size-lesion_size*0.05*0.75-1} and {lesion_size-lesion_size*0.05*0.75+1}')
    
    lesions_r=round_lesions[:lesion_number]
    lesions_nr=not_round_lesions[:lesion_number]
    round_lesions=[[np.array(resize(round_lesions[i][0],(round_lesions[i][0].shape[0]*factor,round_lesions[i][0].shape[1]*factor))),round_lesions[i][1]] for i in range(len(lesions_r))]
    not_round_lesions=[[np.array(resize(not_round_lesions[i][0],(not_round_lesions[i][0].shape[0]*factor,not_round_lesions[i][0].shape[1]*factor))),not_round_lesions[i][1]] for i in range(len(lesions_nr))]
    
    return round_lesions, not_round_lesions

def rescale_values(image,max_val,min_val):
    '''
    image - numpy array
    max_val/min_val - float
    '''
    return (image-image.min())/(image.max()-image.min())*(max_val-min_val)+min_val

def select_coordinates(slice_image, lesions,white_constant,seed):
    '''
    slice_image is the brain slice to use
    lesions is a list of the lesions (with len=number_of_lesions) to use
    colour_lesion is either 'black' or 'white'
    white_constant is the constant that is multiplied with the lesion mask to create lighter or darker lesions
    '''

    np.random.seed(seed)
    
    brain_mask=np.array(slice_image)>0
    brain_mask=brain_mask.astype(float)
    x,y = np.where(brain_mask == 1.)
    
    lesion_brain=slice_image.copy().astype(float)
    lesion_mask=brain_mask.copy() 
    lesion_added=0
    ground_truth=np.zeros(slice_image.shape)
    min_value=0.1 #min value for the lesion intensity 
    max_value=0.9 #max value for the lesion intensity
    brain_image=slice_image.copy()
    
    while lesion_added<len(lesions):
        i=np.random.choice(np.arange(len(x)))
        coordinate=[x[i],y[i]]
        lesion=lesions[lesion_added]
        lesion_rescale = rescale_values(lesion,max_value,min_value)
        lesion_rescale=rescale_values(lesion,white_constant,min_value)
        
        #creating the lesion mask and ground truth
        if (brain_mask[coordinate[0]: coordinate[0] + lesion.shape[0], coordinate[1]: coordinate[1] + lesion.shape[1]] ==1).all():
            #checks if the lesion that will be added is completly in a white space of the lesion mask 
            #(this means that the new lesion is not overlaping an existing one and is completly in the brain area)
            lesion_mask[coordinate[0]: coordinate[0] + lesion.shape[0], coordinate[1]: coordinate[1] + lesion.shape[1]] -= lesion_rescale
            lesion_added+=1
            ground_truth[coordinate[0]: coordinate[0] + lesion.shape[0], coordinate[1]: coordinate[1] + lesion.shape[1]]+=lesion
            brain_mask=lesion_mask

    
    brain_image=slice_image.copy()
    brain_mask=np.array(slice_image)>0
    #creating the white lesions
       
    brain_image[brain_image>0]=1-brain_image[brain_image>0]        
    brain_image[lesion_mask!=0]*=lesion_mask[lesion_mask!=0]
    brain_image[brain_mask]=1-brain_image[brain_mask]
    

    
    return lesion_mask,brain_image,ground_truth
    
def add_lesions(slice_image,round_lesions,not_round_lesions,min_lesion,max_lesion,white_constant,seed,max_brain):
    #for each slice we chose: random number of lesions, random lesions, random coordinates
    
    np.random.seed(seed)
    number_of_lesions=np.random.randint(min_lesion,max_lesion+1)
    lesion_type=np.random.randint(0,2)
    #lesion_type=0 - round
    #lesion_type=1 - not round
    
    #get lesions from type of lesions (and target)
    added=rescale_values(slice_image.copy(),max_brain,0)
    if lesion_type==0: #round
        target=0
        with open('round_lesions.pkl', 'rb') as f:
            lesion_list=pickle.load(f)
        np.random.shuffle(round_lesions)
        
    elif lesion_type==1: #not round
        target=1
        with open('not_round_lesions.pkl', 'rb') as f:
            lesion_list=pickle.load(f) 
        np.random.shuffle(lesion_list)
        
    lesions=[i[0] for i in lesion_list[:number_of_lesions]]
    #add the lesions
    
    lesion_mask,lesion_brain_white,ground_truth=select_coordinates(added, lesions,white_constant,seed)
    
    
    return lesion_mask,lesion_brain_white,ground_truth,target,number_of_lesions

def change_images(image):
    image=np.repeat(image[..., np.newaxis], 3, axis=2)
    image=resize(image, (224, 224))
    image=image.transpose(2,0,1)
    return image

def create_dataset(slices,round_lesions,not_round_lesions,min_lesion=3,max_lesion=5,white_constant=0.85,seed=0,max_brain=1):

    dataset_white=[]
    number_lesions=[]
    lesion_mask_list=[]
    ground_truths=[]
    for slice_idx in range(len(slices)):
        lesion_mask,lesion_brain_white,ground_truth,target,number_of_lesions=add_lesions(slices[slice_idx],
                                                                                         round_lesions,
                                                                                         not_round_lesions,
                                                                                         min_lesion=min_lesion,
                                                                                         max_lesion=max_lesion,
                                                                                         white_constant=white_constant,
                                                                                        seed=seed,
                                                                                        max_brain=max_brain)
        dataset_white.append([change_images(lesion_brain_white),target])
        number_lesions.append(number_of_lesions)
        lesion_mask_list.append(lesion_mask)
        ground_truths.append(ground_truth)
        seed+=1
        
        if slice_idx%1500==0:
            print(f'slice {slice_idx}/{len(slices)} = {round(100*slice_idx/len(slices),2)}%')
        
    return dataset_white,number_lesions,lesion_mask_list,ground_truths
        
    


In [2]:
def boolList2BinString(lst):
    # lst is a binary list that corresponds to the comparison between the predicted labels and the real labels
    # returns a binary string version of the list
    
    return '0b' + ''.join(['1' if x else '0' for x in lst])


def correctly_classified_intersection(to_save_list):
    # to_save_list is a dictionary with the keys: 
    #    model: name of the file where the model is saved
    #    AUROC: AUROC of the correspondig model
    #    ACC: Accuracy of the correspondig model
    #    AUPRC: AUPRC of the correspondig model
    #    real_labels: tensor with the real labels
    #    pred_labels: list of the predicted labels
    #    logits: tensor with the logits predicted by the model
    #    block: degree of finetuning (1, 2, 3, 4 or a)
    #    seed: the seed used to train the model
    #
    # returns the indexes of the images that are correctly classified by every model
    # (the dataset of these images should be the one used to obtain the labels and logits of to_save_list)

    bools = []
    for i in to_save_list:
        compare_list = np.array(i['real_labels'])==np.array(i['pred_labels'])
        bins = int(boolList2BinString(compare_list),2)
        bools.append(bins)

    #obtain the intersection of the correctly classified values using bitwize and between the bins and a list of ones 
    value = '1'
    l=[str(value) for _ in range(len(dataset))]
    res=int('0b'+''.join(l),2)
    for i in bools:
        res = res & i #bitwise and

    # calculate the number of correct 
   
    r='{0:08319b}'.format(res)
    n=0
    for i in r:
        if i=='1':
            n+=1
    
    print(f"Obtained {n} images correctly classified by all the models in {len('{0:08319b}'.format(res))} total images")

    idx=[i for i,x in enumerate(r) if x=='1']
    
        
    return idx
    
    
    
def incorrectly_classified_intersection(to_save_list):
    # to_save_list is a dictionary with the keys: 
    #    model: name of the file where the model is saved
    #    AUROC: AUROC of the correspondig model
    #    ACC: Accuracy of the correspondig model
    #    AUPRC: AUPRC of the correspondig model
    #    real_labels: tensor with the real labels
    #    pred_labels: list of the predicted labels
    #    logits: tensor with the logits predicted by the model
    #    block: degree of finetuning (1, 2, 3, 4 or a)
    #    seed: the seed used to train the model
    #
    # returns the indexes of the images that are correctly classified by every model
    # (the dataset of these images should be the one used to obtain the labels and logits of to_save_list)

    bools = []
    for i in to_save_list:
        compare_list = np.array(i['real_labels'])==np.array(i['pred_labels'])
        bins = int(boolList2BinString(compare_list),2)
        bools.append(bins)

    #obtain the intersection of the correctly classified values using bitwize or between the bins and a list of zeros 
    value = '0'
    l=[str(value) for _ in range(len(dataset))]
    res=int('0b'+''.join(l),2)
    for i in bools:
        res = res | i #bitwise or

    # calculate the number of correct 
    r='{0:08319b}'.format(res)
    n=0
    for i in r:
        if i=='0':
            n+=1
            
    print(f"Obtained {n} images incorrectly classified by all the models in {len('{0:08319b}'.format(res))} total images")

    idx=[i for i,x in enumerate(r) if x=='0']

    return idx


In [3]:
def load_VGG_model(path,device):
    model = models.vgg16(pretrained=True)
    model.classifier=model.classifier[:-1]
    last_layers=[nn.Linear(4096,2)]
    model.classifier = nn.Sequential(*list(model.classifier)+last_layers) 

    model.load_state_dict(torch.load(path,map_location=device))
    
    model.features[1]=nn.ReLU(inplace=False)
    model.features[3]=nn.ReLU(inplace=False)
    model.features[6]=nn.ReLU(inplace=False)
    model.features[8]=nn.ReLU(inplace=False)
    model.features[11]=nn.ReLU(inplace=False)
    model.features[13]=nn.ReLU(inplace=False)
    model.features[15]=nn.ReLU(inplace=False)
    model.features[18]=nn.ReLU(inplace=False)
    model.features[20]=nn.ReLU(inplace=False)
    model.features[22]=nn.ReLU(inplace=False)
    model.features[25]=nn.ReLU(inplace=False)
    model.features[27]=nn.ReLU(inplace=False)
    model.features[29]=nn.ReLU(inplace=False)
    model.classifier[1]=nn.ReLU(inplace=False)
    model.classifier[4]=nn.ReLU(inplace=False)
    
    model.to(device)
    return model

In [40]:
def explanations(model, dataset_img, gt_image, DEVICE):
    # model is the VGG model
    # dataset img is an element of dataset (image,target)
    # gt image is a 224x224 gt image
    test_img, target = dataset_img
    
    transformed_img=torch.tensor(test_img)
    input_image = transformed_img.unsqueeze(0)
    data = input_image.to(DEVICE,dtype=torch.float)


    attribution_gradient = abs(IntegratedGradients(model).attribute(data, target=target).cpu().detach().numpy())
    attribution_gradientshap = abs(GradientShap(model).attribute(data, target=target, baselines=torch.zeros(data.shape).to(DEVICE)).cpu().detach().numpy())
    attribution_deeplift = abs(DeepLift(model).attribute(data, target=target).cpu().detach().numpy())
    attribution_saliency = abs(np.array(torch.Tensor.cpu(Saliency(model).attribute(data, target=target))))
    attribution_InputXGradient=abs(InputXGradient(model).attribute(data, target=target).cpu().detach().numpy())
    attribution_backprop = abs(np.array(torch.Tensor.cpu(GuidedBackprop(model).attribute(data, target=target))))
    attribution_deconv = abs(Deconvolution(model).attribute(data, target=target).cpu().detach().numpy())
    attribution_LRP = abs(LRP(model).attribute(data, target=target).cpu().detach().numpy())


    rgb_weights = [0.2989, 0.5870, 0.1140]
    gt=np.dot(gt_image[...,:3], rgb_weights) 
    
    return attribution_gradient, attribution_gradientshap, attribution_deeplift, attribution_saliency, attribution_InputXGradient, attribution_backprop, attribution_deconv, attribution_LRP


In [5]:
def obtain_derivations_from_confusion_matrix(exp,gt):
    # obtains the tp, tn, fp and fn top p pixels with the highest intensity in exp, where p is the number of pixels !=0 in gt.
    
    p = len(gt[gt>0])
    
    # turn exp into grayscale
    rgb_weights = [0.2989, 0.5870, 0.1140]
    att=exp.squeeze(0).transpose(1,2,0)    
    grayscale_att = np.dot(att[...,:3], rgb_weights)
    
    # create a mask of the pixels with the highest intensity
    sorted_=np.sort(grayscale_att, axis=None)
    sorted_=sorted_[::-1]
    min_att_val=sorted_[p]

    mask=grayscale_att>min_att_val

    tp = mask*(gt>0)
    tp = len(tp[tp > 0])
    
    tn = ~mask*~(gt>0)
    tn = len(tn[tn>0])
    
    fp = mask*~(gt>0)
    fp = len(fp[fp>0])
    
    fn = ~mask*(gt>0)
    fn = len(fn[fn>0])

    return tp, tn, fp, fn

In [6]:
device = 'cuda:2'

def metrics_per_model(model,image_dataset,ground_truths_dataset,device):
    t0 = time.time()
    metrics = {
               'gradient': {'TP':[],'TN':[], 'FN':[], 'FP':[]},
               'gradshap': {'TP':[],'TN':[], 'FN':[], 'FP':[]},
               'deeplift': {'TP':[],'TN':[], 'FN':[], 'FP':[]},
               'saliency': {'TP':[],'TN':[], 'FN':[], 'FP':[]},
               'inputXGrad': {'TP':[],'TN':[], 'FN':[], 'FP':[]},
               'backprop': {'TP':[],'TN':[], 'FN':[], 'FP':[]},
               'deconv': {'TP':[],'TN':[], 'FN':[], 'FP':[]},
               'LRP': {'TP':[],'TN':[], 'FN':[], 'FP':[]}
              }

    xai_methods = ['gradient', 'gradshap', 'deeplift', 'saliency', 'inputXGrad', 'backprop', 'deconv', 'LRP']

    for image_idx in range(len(image_dataset)):
        gt = ground_truths_dataset[image_idx]
        #obtain explanation
        explanations_heatmaps = explanations(model,image_dataset[image_idx],gt,device)
        
        for i, exp in enumerate(explanations_heatmaps):
            
            tp, tn, fp, fn = obtain_derivations_from_confusion_matrix(exp,gt)
            
            metrics[xai_methods[i]]['TP'].append(tp)
            metrics[xai_methods[i]]['TN'].append(tn)
            metrics[xai_methods[i]]['FP'].append(fp)
            metrics[xai_methods[i]]['FN'].append(fn)
            
        if image_idx%500 == 0:
            print(f'{image_idx}/{len(image_dataset)} | {100*round(image_idx/len(image_dataset),2)} %')
            print(f'time elapsed: {(time.time()-t0) // 60:.0f}m {(time.time()-t0) % 60:.0f}s')
            
            
    return metrics

        
    

In [7]:
!gpustat

cuda01                         Wed Mar 29 11:30:49 2023  470.161.03
[0] NVIDIA GeForce GTX 1080 Ti | 55'C, 100 % |  7955 / 11178 MB | jihoon(803M) jihoon(1875M) johannes(5267M) gdm(4M)
[1] NVIDIA GeForce GTX 1080 Ti | 68'C,  11 % |  7823 / 11178 MB | jihoon(613M) jihoon(1905M) johannes(5271M) gdm(4M)
[2] NVIDIA GeForce GTX 1080 Ti | 58'C, 100 % |  6531 / 11178 MB | jihoon(1947M) jihoon(1947M) johannes(2627M) gdm(4M)
[3] NVIDIA GeForce GTX 1080 Ti | 61'C, 100 % |  3904 / 11178 MB | jihoon(1947M) jihoon(1947M) gdm(4M)
[4] NVIDIA GeForce GTX 1080 Ti | 63'C, 100 % |  9439 / 11178 MB | jihoon(2307M) jihoon(1851M) johannes(5271M) gdm(4M)
[5] NVIDIA GeForce GTX 1080 Ti | 55'C,   0 % |  2637 / 11178 MB | johannes(2629M) gdm(4M)
[6] NVIDIA GeForce GTX 1080 Ti | 66'C,   0 % |  5279 / 11178 MB | johannes(5271M) gdm(4M)
[7] NVIDIA GeForce GTX 1080 Ti | 32'C,   0 % |     8 / 11178 MB | gdm(4M)


### Obtaining the intersection of the correctly classified and respective ratios


In [8]:
# creating dataset

seed=0
np.random.seed(seed)
random.seed(seed)
plt.rc('image',cmap='gray')  

start = time.time()

# creating lesions
number_of_lesions=50 #amount of lesions in each lesion list
size_of_lesions=70 #size of all the lesions
factor=2

round_lesions, not_round_lesions=create_lesions(number_of_lesions,size_of_lesions,factor=factor)

done = time.time()
elapsed = done - start
print(f'took {round(elapsed,2)}s')

# loading slices 
with open('slices_validation.pkl', 'rb') as f:
    validation_slices,target_valid_gender,target_valid_age = pickle.load(f)
    
    
# adding lesions to slices
lesion_max_intensity=0.5
max_brain_intensity=0.7

start = time.time()
print(' ====== holdout ====== ')

dataset,_,_,ground_truths=create_dataset(validation_slices,
                                          round_lesions,
                                          not_round_lesions,
                                          min_lesion=3,
                                          max_lesion=5,
                                          white_constant=lesion_max_intensity,
                                          seed=50000,
                                          max_brain=max_brain_intensity)

done = time.time()
elapsed = done - start
print()
print(f'took {round(elapsed,2)}s')
target_w=[i[1] for i in dataset]
ground_truths = [resize(gt, (224, 224)) for gt in ground_truths]

print(f'{len([i for i in target_w if i==1])} slices of target 1 out of {len(target_w)} slices: {round(100*len([i for i in target_w if i==1])/len(target_w),2)} %')
print(f' number of slices: {len(dataset)}')


round lesions: 51/50 === not round lesions 207/50
number of seeds used: 886
the lesions have size between 66.375 and 68.375
took 12.68s
 ====== holdout ====== 
slice 0/8539 = 0.0%
slice 1500/8539 = 17.57%
slice 3000/8539 = 35.13%
slice 4500/8539 = 52.7%
slice 6000/8539 = 70.27%
slice 7500/8539 = 87.83%

took 261.98s
4277 slices of target 1 out of 8539 slices: 50.09 %
 number of slices: 8539


In [9]:
with open('/home/martao/MRI_dataset/2ndTry/new_models_finetune_with_test_slices/best/save/to_save.pkl', 'rb') as f:
    to_save_best_img = pickle.load(f)
    
with open('/home/martao/MRI_dataset/2ndTry/new_models_finetune_with_test_slices/best/save/metrics_best.pkl', 'rb') as f:
    metrics_best = pickle.load(f)
    
folder = '/home/martao/MRI_dataset/2ndTry/new_models_finetune_with_test_slices/best/'

models_names = os.listdir(folder)
models_names = [i for i in models_names if (i[0]=='n' and i not in list(metrics_best.keys()))]

# correctly classified intersection
idx = correctly_classified_intersection(to_save_best_img)

correct_best=[dataset[i] for i in idx]
correct_gt_best=[ground_truths[i] for i in idx]


device = 'cuda:7'
print()

for model_name in models_names:
    print(model_name)
    model = load_VGG_model(folder+model_name,device)
    metrics_per_model_dict = metrics_per_model(model,correct_best,correct_gt_best,device)
    metrics_best[model_name] = metrics_per_model_dict
    
    with open(folder+'save/metrics_best.pkl', 'wb') as f:
        pickle.dump(metrics_best,f)
        
    print()

Obtained 3681 images correctly classified by all the models in 8539 total images

new_finetuning_3conv_0-Copy1.5_img_2500_0.008_116560000.pt


/home/martao/anaconda3/lib/python3.9/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/martao/anaconda3/lib/python3.9/site-packages/captum/attr/_core/deep_lift.py:336: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(
/home/martao/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:749: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/home/martao/anaconda3/lib/python3.9/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:64: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed 

0/3681 | 0.0 %
time elapsed: 0m 1s
500/3681 | 14.000000000000002 %
time elapsed: 5m 22s
1000/3681 | 27.0 %
time elapsed: 10m 47s
1500/3681 | 41.0 %
time elapsed: 16m 7s
2000/3681 | 54.0 %
time elapsed: 21m 32s
2500/3681 | 68.0 %
time elapsed: 26m 56s
3000/3681 | 81.0 %
time elapsed: 32m 18s
3500/3681 | 95.0 %
time elapsed: 37m 39s

new_finetuning_all_0-Copy1.5_img_2500_0.004_116560000.pt
0/3681 | 0.0 %
time elapsed: 0m 1s
500/3681 | 14.000000000000002 %
time elapsed: 5m 20s
1000/3681 | 27.0 %
time elapsed: 10m 38s
1500/3681 | 41.0 %
time elapsed: 15m 58s
2000/3681 | 54.0 %
time elapsed: 21m 19s
2500/3681 | 68.0 %
time elapsed: 26m 39s
3000/3681 | 81.0 %
time elapsed: 31m 59s
3500/3681 | 95.0 %
time elapsed: 37m 20s

new_finetuning_2conv_0.5_MRI_new_2500_0.03_18464.pt
0/3681 | 0.0 %
time elapsed: 0m 1s
500/3681 | 14.000000000000002 %
time elapsed: 5m 21s
1000/3681 | 27.0 %
time elapsed: 10m 40s
1500/3681 | 41.0 %
time elapsed: 16m 0s
2000/3681 | 54.0 %
time elapsed: 21m 22s
2500/3681 | 

In [9]:
with open('/home/martao/MRI_dataset/2ndTry/new_models_finetune_with_test_slices/same/save/to_save.pkl', 'rb') as f:
    to_save_same_img = pickle.load(f)
    
with open('/home/martao/MRI_dataset/2ndTry/new_models_finetune_with_test_slices/same/save/metrics_same.pkl', 'rb') as f:
    metrics_same = pickle.load(f)  

folder = '/home/martao/MRI_dataset/2ndTry/new_models_finetune_with_test_slices/same/'
models_names = os.listdir(folder)
models_names = [i for i in models_names if (i[0]=='n' and i not in list(metrics_same.keys()))]

# correctly classified intersection
idx = correctly_classified_intersection(to_save_same_img)

correct_same=[dataset[i] for i in idx]
correct_gt_same=[ground_truths[i] for i in idx]

device = 'cuda:7'
print()

for model_name in models_names:
    print(model_name)
    model = load_VGG_model(folder+model_name,device)
    metrics_per_model_dict = metrics_per_model(model,correct_same,correct_gt_same,device)
    metrics_same[model_name] = metrics_per_model_dict
    
    with open(folder+'save/metrics_same.pkl', 'wb') as f:
        pickle.dump(metrics_same,f)
        
    print()

Obtained 3350 images correctly classified by all the models in 8539 total images

new_finetuning_3conv_0.5_MRI_new_2500_0.005_55168461.pt


/home/martao/anaconda3/lib/python3.9/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/martao/anaconda3/lib/python3.9/site-packages/captum/attr/_core/deep_lift.py:336: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(
/home/martao/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:749: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/home/martao/anaconda3/lib/python3.9/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:64: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed 

0/3350 | 0.0 %
time elapsed: 0m 1s
500/3350 | 15.0 %
time elapsed: 5m 32s
1000/3350 | 30.0 %
time elapsed: 11m 5s
1500/3350 | 45.0 %
time elapsed: 16m 39s
2000/3350 | 60.0 %
time elapsed: 22m 14s
2500/3350 | 75.0 %
time elapsed: 27m 46s
3000/3350 | 90.0 %
time elapsed: 33m 18s

new_finetuning_2conv_0-Copy1.5_img_2500_0.0018_5484646.pt
0/3350 | 0.0 %
time elapsed: 0m 1s
500/3350 | 15.0 %
time elapsed: 5m 34s
1000/3350 | 30.0 %
time elapsed: 11m 7s
1500/3350 | 45.0 %
time elapsed: 16m 38s
2000/3350 | 60.0 %
time elapsed: 22m 5s
2500/3350 | 75.0 %
time elapsed: 27m 36s
3000/3350 | 90.0 %
time elapsed: 33m 2s

new_finetuning_3conv_0-Copy1.5_img_2500_0.0008_18464876.pt
0/3350 | 0.0 %
time elapsed: 0m 1s
500/3350 | 15.0 %
time elapsed: 5m 29s
1000/3350 | 30.0 %
time elapsed: 10m 59s
1500/3350 | 45.0 %
time elapsed: 16m 28s
2000/3350 | 60.0 %
time elapsed: 21m 57s
2500/3350 | 75.0 %
time elapsed: 27m 28s
3000/3350 | 90.0 %
time elapsed: 33m 1s

new_finetuning_2conv_0.5_MRI_new_2500_0.01_11656

In [23]:
folder_best = '/home/martao/MRI_dataset/2ndTry/new_models_finetune_with_test_slices/best/save/'

with open(folder_best + 'to_save.pkl', 'rb') as f:
    to_save_best_img = pickle.load(f)

idx = correctly_classified_intersection(to_save_best_img)

correct_best=[dataset[i] for i in idx]
correct_gt_best=[ground_truths[i] for i in idx]

folder_same = '/home/martao/MRI_dataset/2ndTry/new_models_finetune_with_test_slices/same/save/'

with open(folder_same + 'to_save.pkl', 'rb') as f:
    to_save_same_img = pickle.load(f)

idx = correctly_classified_intersection(to_save_same_img)

correct_same=[dataset[i] for i in idx]
correct_gt_same=[ground_truths[i] for i in idx]

Obtained 3681 images correctly classified by all the models in 8539 total images
Obtained 3350 images correctly classified by all the models in 8539 total images


In [24]:
#sobel and laplace
from scipy import ndimage, misc

def obtain_edge_filters(dataset,ground_truth):
    t0 = time.time()
    laplace_metircs = {'TP':[],'TN':[], 'FN':[], 'FP':[]}
    sobel_metircs = {'TP':[],'TN':[], 'FN':[], 'FP':[]}
    
    for i in range(len(dataset)):
        image=dataset[i][0]
        gt=ground_truth[i]
        
        expl_sobel = np.array([ndimage.sobel(image)])
        expl_laplace = np.array([ndimage.laplace(image)])
        
        
        metrics_sobel = obtain_derivations_from_confusion_matrix(expl_sobel, gt)
        metrics_laplace = obtain_derivations_from_confusion_matrix(expl_laplace, gt)
        
        laplace_metircs['TP'].append(metrics_laplace[0])
        laplace_metircs['TN'].append(metrics_laplace[1])
        laplace_metircs['FP'].append(metrics_laplace[2])
        laplace_metircs['FN'].append(metrics_laplace[3])
        
        sobel_metircs['TP'].append(metrics_laplace[0])
        sobel_metircs['TN'].append(metrics_laplace[1])
        sobel_metircs['FP'].append(metrics_laplace[2])
        sobel_metircs['FN'].append(metrics_laplace[3])
        
        if i%500 == 0:
            print(f'{i}/{len(dataset)} | {100*round(i/len(dataset),2)} %')
            print(f'time elapsed: {(time.time()-t0) // 60:.0f}m {(time.time()-t0) % 60:.0f}s')
        
    return laplace_metircs, sobel_metircs



In [28]:
print(' ====== SAME PERFORMANCE ====== ')
laplace_same, sobel_same = obtain_edge_filters(correct_same, correct_gt_same)

with open(folder_same + 'laplace.pkl', 'wb') as f:
    pickle.dump(laplace_same,f)
with open(folder_same + 'sobel.pkl', 'wb') as f:
    pickle.dump(sobel_same,f)

print(' ====== BEST PERFORMANCE ====== ')
laplace_best, sobel_best = obtain_edge_filters(correct_best, correct_gt_best)

with open(folder_best + 'laplace.pkl', 'wb') as f:
    pickle.dump(laplace_best,f)
with open(folder_best + 'sobel.pkl', 'wb') as f:
    pickle.dump(sobel_best,f)

 ====== SAME PERFORMANCE ====== 
0/3350 | 0.0 %
time elapsed: 0m 0s
500/3350 | 15.0 %
time elapsed: 0m 10s
1000/3350 | 30.0 %
time elapsed: 0m 19s
1500/3350 | 45.0 %
time elapsed: 0m 29s
2000/3350 | 60.0 %
time elapsed: 0m 39s
2500/3350 | 75.0 %
time elapsed: 0m 49s
3000/3350 | 90.0 %
time elapsed: 0m 58s
 ====== BEST PERFORMANCE ====== 
0/3681 | 0.0 %
time elapsed: 0m 0s
500/3681 | 14.000000000000002 %
time elapsed: 0m 10s
1000/3681 | 27.0 %
time elapsed: 0m 20s
1500/3681 | 41.0 %
time elapsed: 0m 29s
2000/3681 | 54.0 %
time elapsed: 0m 39s
2500/3681 | 68.0 %
time elapsed: 0m 49s
3000/3681 | 81.0 %
time elapsed: 0m 59s
3500/3681 | 95.0 %
time elapsed: 1m 8s


In [56]:
!gpustat

cuda01                         Wed Mar 29 15:49:25 2023  470.161.03
[0] NVIDIA GeForce GTX 1080 Ti | 52'C, 100 % |  6930 / 11178 MB | jihoon(1875M) johannes(571M) jihoon(499M) johannes(3975M) gdm(4M)
[1] NVIDIA GeForce GTX 1080 Ti | 58'C,   0 % |  8620 / 11178 MB | jihoon(1905M) johannes(6681M) gdm(4M)
[2] NVIDIA GeForce GTX 1080 Ti | 54'C,   0 % |     8 / 11178 MB | gdm(4M)
[3] NVIDIA GeForce GTX 1080 Ti | 59'C,   0 % |  6687 / 11178 MB | johannes(6679M) gdm(4M)
[4] NVIDIA GeForce GTX 1080 Ti | 58'C, 100 % |  5836 / 11178 MB | jihoon(1851M) johannes(3975M) gdm(4M)
[5] NVIDIA GeForce GTX 1080 Ti | 50'C,  48 % |  2835 / 11178 MB | johannes(2827M) gdm(4M)
[6] NVIDIA GeForce GTX 1080 Ti | 60'C,   0 % |  2639 / 11178 MB | johannes(2631M) gdm(4M)
[7] NVIDIA GeForce GTX 1080 Ti | 46'C,   0 % | 11033 / 11178 MB | martao(11025M) gdm(4M)


In [57]:
# random model
device = 'cuda:2'
random_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [58]:

torch.manual_seed(seed)
random_model = models.vgg16(pretrained=False)
random_model.classifier=random_model.classifier[:-1]
last_layers=[nn.Linear(4096,2)]
random_model.classifier = nn.Sequential(*list(random_model.classifier)+last_layers) 

random_model.to(device)

random_model.features[1]=nn.ReLU(inplace=False)
random_model.features[3]=nn.ReLU(inplace=False)
random_model.features[6]=nn.ReLU(inplace=False)
random_model.features[8]=nn.ReLU(inplace=False)
random_model.features[11]=nn.ReLU(inplace=False)
random_model.features[13]=nn.ReLU(inplace=False)
random_model.features[15]=nn.ReLU(inplace=False)
random_model.features[18]=nn.ReLU(inplace=False)
random_model.features[20]=nn.ReLU(inplace=False)
random_model.features[22]=nn.ReLU(inplace=False)
random_model.features[25]=nn.ReLU(inplace=False)
random_model.features[27]=nn.ReLU(inplace=False)
random_model.features[29]=nn.ReLU(inplace=False)
random_model.classifier[1]=nn.ReLU(inplace=False)
random_model.classifier[4]=nn.ReLU(inplace=False)

In [ ]:
same_random_explanations = metrics_per_model(random_model,correct_same,correct_gt_same,device)

with open(folder_same + 'same_random_explanations.pkl', 'wb') as f:
    pickle.dump(same_random_explanations,f)

best_random_explanations = metrics_per_model(random_model,correct_best,correct_gt_best,device)


with open(folder_best + 'best_random_explanations.pkl', 'wb') as f:
    pickle.dump(best_random_explanations,f)

0/3350 | 0.0 %
time elapsed: 0m 1s
